## Inversion function

We invert between conservative and "fluid"* variables. 

In [1]:
import Pkg; 
Pkg.activate("Conf_Fluids")
Pkg.add("Symbolics")
using Symbolics
using StaticArrays

 Activating environment at `~/Julia/Fluidos_Conformes/Conf_Fluids/Project.toml`
   Updating registry at `~/.julia/registries/General`
######################################################################### 100.0%
  Resolving package versions...
No Changes to `~/Julia/Fluidos_Conformes/Conf_Fluids/Project.toml`
No Changes to `~/Julia/Fluidos_Conformes/Conf_Fluids/Manifest.toml`


### The Function to invert

We first define the function to invert. 
We define it first as a numerical function. Then we promote it to a symbolic one and compute its Jacobian. 

    Notice that here $\mu \to -\mu$ we then undo when defining the final maps

    Notice that this can still be optimized in many ways... 

In [53]:
function F(flu,con,χ)
    #the use of a dictionary can go away in any case.
    f = Dict(:μ => 1, :v => 2, :x1 => 3, :x2 => 4,:x3 => 5) #para acordarse de que campo asignamos a cada componente del vector
    c = Dict(:e => 1, :s => 2, :c1 => 3, :c2 => 4,:c3 => 5)

    μ = flu[f[:μ]]  # esto es -μ
    #μ = view(flu,1)
    T = (μ)^(-1//2) # use μ positive, so I changed μ -> -μ
    v = flu[f[:v]]
    χ₀ = χ[1]
    χ₁ = χ[2]
    χ₂ = χ[3]
    x1 = flu[f[:x1]]
    x2 = flu[f[:x2]]
    x3 = flu[f[:x3]]
    γ = (1 - v^2)^(-1//2)
    τ = -2χ₁ * x3 * T / (γ*μ^3) - 24χ₂*(1//2*(1-v^2)x3^2 + 14//3 * x2^2 + 7/5*(1-v^2)x1*x3)/μ^5
    ρ = -6χ₀ / μ^2 - 6χ₁*x1/(γ * μ^4 * T) - 42χ₂*(6//5 *x1^2 + 10γ^2*x2^2 + 3//2*(v^2-1)^2*x3^2)/(μ^5 * γ^2)
    Q = -10χ₀ * x2 * T / μ^3 - 168χ₂ * x2 * (x1 - (v^2 - 1)x3)/(γ * μ^5)

    A = (-12 * χ₂/μ^4)*[3(2γ^2 - 1)  3v*(6γ^2 - 1)    3v^2 ;
                v*(6γ^2 - 1)  (6γ^2*(1 + 2v^2) - 1)  v*(v^2 + 2) ;
                (6γ^2 * v^2 + 1)  v*(6γ^2*(2 + v^2) - 1)  (2v^2 + 1)]

    e = 4//3*ρ*(γ^2 - 1//4) + 2v*γ*Q + v^2*τ
    s = 4//3*ρ*γ^2*v + (v^2+1//1)γ*Q + v*τ;

    Y1 = -3χ₁*γ/T/μ^3 *(2γ^2 - 1//1)
    Y2 = -3χ₁*γ/T/μ^3 *v*(6γ^2 - 1//1)
    Y3 = -3χ₁*γ/T/μ^3 *(6γ^2*v^2 + 1//1)
    
    return  [- con[c[:e]] + 4//3* ρ*(γ^2 - 1//4) + 2//1* Q*γ*v + τ*v^2;
             - con[c[:s]] + 4//3* ρ*γ^2*v + Q*γ*(v^2 + 1//1) + τ*v;
             - con[c[:c1]] + A[1,1]*x1 + A[1,2]*x2 + A[1,3]*x3 + Y1;
             - con[c[:c2]] + A[2,1]*x1 + A[2,2]*x2 + A[2,3]*x3 + Y2;
             - con[c[:c3]] + A[3,1]*x1 + A[3,2]*x2 + A[3,3]*x3 + Y3]
end

F (generic function with 1 method)

Para testear usamos: 
`
flu = [10.0; 0.2; 1.1; 1.5; 1.2]
con = [0.1366314976448222; 0.07009306769467444; 0.06115332989597844; 0.07178418128379448; 0.04927907295689818]
chi = [-1.0; -1.0; -5.0]
`

**Ojo que hay que cambiar el signo a $\mu$!**

In [54]:
flu = [10.0; 0.2; 1.1; 1.5; 1.2]
con = [0.1366314976448222; 0.07009306769467444; 0.06115332989597844; 0.07178418128379448; 0.04927907295689818]
chi = [-1.0; -1.0; -5.0]
@time F(flu,con,chi)

  0.146915 seconds (392.21 k allocations: 20.885 MiB)


5-element Array{Float64,1}:
 -1.6263032587282567e-18
 -1.3877787807814457e-17
 -3.469446951953614e-18
  3.469446951953614e-18
  1.734723475976807e-18

Once we are sure about the function we promote it to a **symbolic function** and compute the Jacobian 

In [159]:
N=5

@variables f[1:N], c[1:N], p[1:3]

5

In [6]:
F(f,c,p);

In [7]:
JS = Symbolics.jacobian(F(f,c,p),f);

In [8]:
J_exp = Symbolics.build_function(JS, f, c, p);

In [9]:
Jac = eval(J_exp[1]);

In [10]:
Jac(rand(5),rand(5),rand(3)) #check that it returns a metric

5×5 Array{Float64,2}:
 2.1326e8   -7.36384e6       -2.47609e6  -1.29663e7      -1.62161e6
 1.91586e8  -7.38982e6       -2.37722e6  -1.10918e7      -1.64112e6
 7.87715e6  -8.70425e5  -107976.0        -2.51339e5  -18105.2
 7.43887e6  -8.45607e5   -83779.5        -2.53325e5  -24170.9
 6.54349e6  -8.1213e5    -72674.7        -2.10055e5  -29720.8

One can make the inversion in a symbolic way, but it is a very long expression, so probably takes more time to evaluate than to invert.

Now we define the Newton-Raphson step.

In [11]:
function NR_step!(F, Jac, u0, y, p)
    u = u0 - Jac(u0,y, p) \ F(u0,y,p)
end

NR_step! (generic function with 1 method)

To check we give an arbitrary vector u0, and use the equation to compute y (using y=0 in F). We then perturbe u0 use the scheme to get u0 again.

In [170]:
p = [-1.0; -1.0; -5.0]
u0 = @SVector[1.0, 0.30, 2.0, 4.0, 5.0]
y = F(u0,[0,0,0,0,0],p)

F(u0, y ,p)

5-element Array{Float64,1}:
  5.002220859751105e-12
  0.0
 -2.6201263381153694e-14
  1.127986593019159e-13
  2.930988785010413e-14

In [176]:
u0 = @SVector[1.0, 0.30, 2.0, 4.0, 5.0] + 0.1*rand(5)
#u0 = [1.0; 0.30; 3.0; 4.0; 3.0]
for i ∈ 1:25
    u0 = NR_step!(F, Jac, u0, y, p)
end
println(u0)
println()
println(F(u0,y,p))
println()
println("error = $(sqrt(F(u0,y,p)'*F(u0,y,p)))")

[1.0, 0.29999999999999993, 2.0000000000000013, 4.000000000000001, 5.0]

[2.1373125491663814e-11, 4.547473508864641e-12, 3.717026686445024e-13, 7.94031507211912e-13, 2.5490720645393594e-13]

error = 2.1870611221190414e-11


We see that it is working correctly with 25 iterations.
Now we define a vector of variables, like the case when we are solving the pde equation. Something is wrong when trying to use static arrays here, don't know way. But in any case it is fast.

In [182]:
M = 10^3
ui = zeros(N,M)
# it does not work with SMatrix
for j ∈ 1:M
        ui[:,j] = [10.0; 0.2; 1.1; 1.5; 1.2] + 0.1*rand(5)
end
#u0 = SMatrix{N,M}([(i+j)/(N+M) for i ∈ 1:N, j ∈ 1:M])
#u0 = @SMatrix rand(N,M)
#yy = @SMatrix rand(N,M)
#yy = SMatrix{N,M}([0. for i ∈ 1:N, j ∈ 1:M])
yy = zeros(N,M)
for j ∈ 1:M
    yy[:,j] = F(ui[:,j],[0,0,0,0,0],p)
#    yy[:,j] = convert(Array{Float64,1},F(u0[:,j],yy[:,j],p))
end

In [183]:
j = 200
F(ui[:,j],yy[:,j], p)'*F(ui[:,j],yy[:,j], p)

2.715862148139813e-34

In [184]:
u0 = (ui .- 0.0001 .*rand(N,M))

5×1000 Array{Float64,2}:
 10.0982    10.0036    10.0639    …  10.0268    10.0079    10.0453
  0.224099   0.208545   0.270358      0.244236   0.215885   0.241525
  1.19602    1.18512    1.11974       1.10932    1.18628    1.1795
  1.58214    1.49999    1.53093       1.58623    1.57069    1.58827
  1.29732    1.25374    1.28725       1.25256    1.2229     1.21069

Notice that we square tol for we use it on the square of the error. 


In [186]:
tol = 10^(-8)
tol = tol^2
iter_max = 5

@time begin
for j ∈ 1:M
    iter = 1
    while F(u0[:,j],yy[:,j], p)'*F(u0[:,j],yy[:,j], p) > tol && iter < iter_max
        u0[:,j] = NR_step!(F, Jac, u0[:,j], yy[:,j], p)
        iter = iter + 1
        if iter == iter_max 
            println("iter_max reached j = $j")
        end
    end
    #println(iter)
end
end

  0.009951 seconds (53.94 k allocations: 4.104 MiB)


Now that we have checked all parts we define the final functions:

In [192]:
"""
    c_to_f!(flu, con, p)

Compute the fluid variables `flu` from the conservative `con` ones.
It inverts the function `F(flu,con,χ) = con(flu,χ) - con` using 
Newto-Raphson method.

As parameters have: 
    χ = Function parameters, 
    tol = error tolerance for the inversion, 
    iter_max = maximum iteration number for the NR method, 
    N = number of variables (5 here), 
    M = dimension of vector of variables

# Examples
```julia-repl
julia> 
tol = 10^(-16)
iter_max = 40
χ = [-1.0; -1.0; -5.0]
M=1
N=5
p = (χ, tol, iter_max, N, M)
flu = [-10.0; 0.2; 1.1; 1.5; 1.2]
con = [0.1366314976448222; 0.07009306769467444; 0.06115332989597844; 0.07178418128379448; 0.04927907295689818]
c_to_f!(flu + 0.1* rand(N), con, p) - con
5×1 Array{Float64,2}:
  2.1953994178147695e-11
 -1.393440918207034e-12
 -4.0323300254385686e-13
 -1.3405498933138915e-11
  1.2895684520231043e-11
```
"""
function c_to_f!(flu, con, p)
    χ, tol, iter_max, N, M = p
    for j ∈ 1:M
        flu[1,j] = -flu[1,j]
        iter = 1
        while F(flu[:,j],con[:,j], χ)'*F(flu[:,j],con[:,j], χ) > tol && iter < iter_max
            flu[:,j] = NR_step!(F, Jac, flu[:,j], con[:,j], χ)
            iter = iter + 1
            if iter == iter_max 
            println("iter_max reached j = $j")
            end
        end
    #println(iter)
        flu[1,j] = -flu[1,j]
    end
    return flu[:,:]
end

c_to_f!

In [195]:
?c_to_f!;

In [187]:
tol = 10^(-16)
iter_max = 40
χ = [-1.0; -1.0; -5.0]
M=1
N=5
p = (χ, tol, iter_max, N, M)
flu = [-10.0; 0.2; 1.1; 1.5; 1.2]
con = [0.1366314976448222; 0.07009306769467444; 0.06115332989597844; 0.07178418128379448; 0.04927907295689818]
flu_i = flu + 0.1*rand(5);

In [188]:
@time c_to_f!(flu_i, con, p) - flu

  0.001815 seconds (922 allocations: 75.406 KiB)


5×1 Array{Float64,2}:
  2.1953994178147695e-11
 -1.393440918207034e-12
 -4.0323300254385686e-13
 -1.3405498933138915e-11
  1.2895684520231043e-11

In [197]:
"""
    f_to_c!(flu, con, p)

Compute the conservative `con` variables for the fluid ones `flu`.
It uses the function `F(flu,con,χ) = con(flu,χ) - con` with `con = 0`.

As parameters have: 
    χ = Function parameters
    N = number of variables (5 here), 
    M = dimension of vector of variables

# Examples
```julia-repl
julia> 
χ = [-1.0; -1.0; -5.0]
M=1
N=5
p = (χ, N, M)
flu = [-10.0; 0.2; 1.1; 1.5; 1.2]
con = [0.1366314976448222; 0.07009306769467444; 0.06115332989597844; 0.07178418128379448; 0.04927907295689818]
f_to_c!(flu,con+rand(N),p) - con
5×1 Array{Float64,2}:
  0.0
 -1.3877787807814457e-17
  0.0
  0.0
  0.0
```
"""
function f_to_c!(flu, con, p)
    χ, N, M = p
    y = zeros(N)
    for j ∈ 1:M
        flu[1,j] = -flu[1,j]
        con[:,j] = F(flu[:,j],y, χ)
    end
    return con[:,:]
end

f_to_c!

In [196]:
N=5
M=1
p = (χ, N, M)
χ = [-1.0; -1.0; -5.0]
flu = [-10.0; 0.2; 1.1; 1.5; 1.2]
con = [0.1366314976448222; 0.07009306769467444; 0.06115332989597844; 0.07178418128379448; 0.04927907295689818]
f_to_c!(flu,con+rand(N),p) - con

5×1 Array{Float64,2}:
  0.0
 -1.3877787807814457e-17
  0.0
  0.0
  0.0